In [22]:
from datetime import date
import datetime

In [ ]:
import pandas as pd
import geopandas as gpd
from h3 import h3
import sys
from pandas.io.sql import table_exists
import psycopg2
import psycopg2.extras as extras
# import heroku_config as config
import os
import dbutils as dbutils
from sqlalchemy import create_engine

In [3]:

def lat_lng_to_h3(row, h3_level=4):
    return h3.geo_to_h3(row.geometry.y, row.geometry.x, h3_level)



def load_data():

    emissions_df = pd.DataFrame()
    cols = ['dates', 'h3', 'lat', 'lng']
    products = ['methane', 'carbonmonoxide', 'ozone', 'nitrogendioxide']

    for product in products:
        #limit 10000 because Heroku, pulling 2500 rows for each product
        url = f'https://api.v2.emissions-api.org/api/v2/{product}/geo.json?country=IND&begin=2021-01-01&end=2021-11-11&limit=2500&offset=0'

        print(f'Pulling records for {product}')
        product_data = gpd.read_file(url)
        print('Pull success')

        product_data['lng'] = product_data.geometry.x
        product_data['lat'] = product_data.geometry.y
        product_data['timestamp'] = pd.to_datetime(product_data['timestamp'])
        product_data['h3'] = product_data.apply(lat_lng_to_h3, axis=1)
        product_data['dates'] = pd.to_datetime(product_data['timestamp']).dt.date
        
        daily_data = product_data.groupby(['h3', 'dates'], as_index=False).mean('value')
        daily_data.set_index(['dates'])
        daily_data = daily_data.rename(columns={'value':f'{product}'})

        # cols = ['dates', 'h3', 'lat', 'lng']
        if emissions_df.empty:
            cols.extend([f'{product}'])
            emissions_df = daily_data.copy()
        else:
            cols.extend([f'{product}'])
            emissions_df = emissions_df.merge(daily_data[['dates','h3', f'{product}']], on=['dates', 'h3'])[cols]

#     emissions_df.to_sql('emissions', con = engine, if_exists='append')
    return

# DATABASE_URL = os.environ.get('DATABASE_URL')
# engine = create_engine(DATABASE_URL, echo = False)

# first_row = engine.execute('SELECT * FROM emissions').fetchone()
# print(first_row)
# if first_row is None:
#     print('Populating data')
#     load_data(engine)
#     print('Populate successful')


In [29]:
emissions_df = pd.DataFrame()
cols = ['dates', 'h3', 'lat', 'lng']
products = ['methane']
# products = ['methane', 'carbonmonoxide', 'ozone', 'nitrogendioxide']

days = 10
date0 = datetime.datetime.today() - datetime.timedelta(days=days) #n=1
begin = str(date0).split(' ')[0]
end = str(date.today())

for product in products:
    #limit 10000 because Heroku, pulling 2500 rows for each product
    url = f'https://api.v2.emissions-api.org/api/v2/{product}/geo.json?country=IND&begin={begin}&end={end}&limit=2500&offset=0'

    print(f'Pulling records for {product}')
    product_data = gpd.read_file(url)
    print('Pull success')

    product_data['lng'] = product_data.geometry.x
    product_data['lat'] = product_data.geometry.y
    product_data['timestamp'] = pd.to_datetime(product_data['timestamp'])
    product_data['h3'] = product_data.apply(lat_lng_to_h3, axis=1)
    product_data['dates'] = pd.to_datetime(product_data['timestamp']).dt.date

    daily_data = product_data.groupby(['h3', 'dates'], as_index=False).mean('value')
    daily_data.set_index(['dates'])
    daily_data = daily_data.rename(columns={'value':f'{product}'})

    # cols = ['dates', 'h3', 'lat', 'lng']
    if emissions_df.empty:
        cols.extend([f'{product}'])
        emissions_df = daily_data.copy()
    else:
        cols.extend([f'{product}'])
        emissions_df = emissions_df.merge(daily_data[['dates','h3', f'{product}']], on=['dates', 'h3'])[cols]

Pulling records for methane
Pull success


In [32]:
emissions_df

,h3,dates,methane,lng,lat
0,843c101ffffffff,2022-02-01,1927.048340,84.269619,26.055117
1,843c105ffffffff,2022-02-01,1931.592896,84.025744,25.700012
2,843c107ffffffff,2022-02-01,1929.404053,84.483148,25.707295
3,843c10dffffffff,2022-02-01,1931.219727,83.812326,26.046654
4,843c111ffffffff,2022-02-01,1947.608643,85.435991,26.421230
...,...,...,...,...,...
1885,8461a97ffffffff,2022-02-04,1923.820312,79.398143,10.704351
1886,8461ab1ffffffff,2022-02-04,1942.899170,79.834672,11.437967
1887,8461ab3ffffffff,2022-02-04,1921.112915,79.414967,11.445860
1888,8461ab7ffffffff,2022-02-04,1919.220093,79.633311,11.813054


In [37]:
emissions_df.groupby('h3')#['methane'].transform(lambda x: (x - x.min()) / x.max()-x.min())

In [38]:
df = emissions_df.copy()

In [47]:
cols = 'methane'
g = df.groupby('h3')
min_, max_ = g[cols].transform('min'), g[cols].transform('max')
df[cols + '_scale'] = (df[cols] - min_) / (max_ - min_)

In [54]:
min_ = df['methane'].min()
max_ = df['methane'].max()

In [55]:
min_, max_

(1834.533935546875, 1972.8651123046875)

In [56]:
df['nmethane'] = (df['methane']-min_)/(max_-min_)

In [57]:
df

,h3,dates,methane,lng,lat,methane_scale,nmethane
0,843c101ffffffff,2022-02-01,1927.048340,84.269619,26.055117,NaN,0.668789
1,843c105ffffffff,2022-02-01,1931.592896,84.025744,25.700012,NaN,0.701642
2,843c107ffffffff,2022-02-01,1929.404053,84.483148,25.707295,NaN,0.685819
3,843c10dffffffff,2022-02-01,1931.219727,83.812326,26.046654,NaN,0.698944
4,843c111ffffffff,2022-02-01,1947.608643,85.435991,26.421230,NaN,0.817420
...,...,...,...,...,...,...,...
1885,8461a97ffffffff,2022-02-04,1923.820312,79.398143,10.704351,NaN,0.645454
1886,8461ab1ffffffff,2022-02-04,1942.899170,79.834672,11.437967,NaN,0.783375
1887,8461ab3ffffffff,2022-02-04,1921.112915,79.414967,11.445860,NaN,0.625882
1888,8461ab7ffffffff,2022-02-04,1919.220093,79.633311,11.813054,NaN,0.612199
